<a href="https://colab.research.google.com/github/ddhackiisc/code/blob/master/DILI/DILI_predict_ResNet/Resnet_Adwaith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import AveragePooling1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.convolutional import ZeroPadding1D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input 
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
class Resnet:
  @staticmethod
  def residual_module(data, K ,stride, chanDim=-1, red=False, reg=0.0001, bnEps=2e-5, bnMom=0.9, i=-1, j=-1):
    shortcut = data #the shortcut branch of a residual module is initialized as the input
 
    #First block of resudual module
    bn1 = BatchNormalization(epsilon=bnEps, momentum=bnMom, name='bn1_'+str(i)+'_'+str(j))(data)
    act1 = Activation("relu",name='act1_'+str(i)+'_'+str(j))(bn1)
    conv1 = Conv1D(K, 9, use_bias= False, kernel_regularizer=l2(reg), padding='same', name='conv1_'+str(i)+'_'+str(j))(act1)
 
    #2nd block of residual module
    bn2 = BatchNormalization(epsilon=bnEps, momentum=bnMom, name='bn2_'+str(i)+'_'+str(j))(conv1)
    act2 = Activation("relu", name='act2_'+str(i)+'_'+str(j))(bn2)
    conv2 = Conv1D(K, 9, use_bias= False, kernel_regularizer=l2(reg),padding='same', name='conv2_'+str(i)+'_'+str(j))(act2)
 
    #In the beginning of each stage of the resnet you will have adjust the shortcut dimensions
    if red:
      shortcut = Conv1D(K, 9, use_bias=False, kernel_regularizer=l2(reg), padding='same', name='conv_srt_'+str(i)+'_'+str(j))(act1)
 
    x = add([conv2, shortcut], name='add_'+str(i)+'_'+str(j)) #add the second residual moduleand the shortcut
 
    return x
 
  @staticmethod
  def build(input_shape, classes, stages, filters, reg=0.0001, bnEps=2e-5, bnMom=0.9):
    #create input and apply BN
    inputs = Input(shape=input_shape)
    x = BatchNormalization(epsilon=bnEps, momentum=bnMom, name='bn_input')(inputs)
 
    #Create the starting conv layer
    x = Conv1D(filters[0], 9, use_bias=False, padding='same', kernel_regularizer=l2(reg), name='conv_first_layer')(x)
    x = BatchNormalization(epsilon=bnEps, momentum=bnMom, name='bn_first_layer')(x)
    x = Activation('relu', name='act_first_layer')(x)
    
 
    #loop over the stages
    for i in range(0, len(stages)):
      stride  = 1
      #first layer in each stage should reduce the dimensions of input 
      x = Resnet.residual_module(x, filters[i+1], stride, red=True, bnEps=bnEps, bnMom=bnMom, i=i+1, j=1)
 
      for j in range(0, stages[i] -1):
        #apply a renset module
        x = Resnet.residual_module(x, filters[i+1], stride, bnEps=bnEps, bnMom=bnMom, i=i+1, j=j+2)
    
    x = BatchNormalization(epsilon=bnEps, momentum=bnMom, name='bn_last_layer')(x)
    x = Activation("relu")(x)
 
    #Final dense layer applied with softmax activation
    x = Dense(970, kernel_regularizer=l2(reg), name='out_dense_layer')(x)
    x = Dense(classes, activation='sigmoid', name='last_layer', kernel_regularizer=l2(reg))(x)
    #x = Activation('softmax')(x)
    # x = Activation('sigmoid', name='out_act')(x)
 
 
    #create the model
    model = Model(inputs, x, name='resnet')
 
    #return the model
    return model

In [ ]:
stages = []
filters = [9]

In [ ]:
my_model = Resnet.build((881,1), 1, stages, filters)

In [ ]:
my_model.summary()

In [ ]:
from keras.optimizers import Adam 


# my_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'] )
my_model.compile(optimizer=Adam(learning_rate=0.0001), loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'] )

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard
 
from datetime import datetime
from packaging import version
!rm -rf ./logs/

In [ ]:
train_df = pd.read_csv("combined_train.csv")
train_smiles = train_df['smiles'].to_numpy()
train_labels = train_df['label'].to_numpy()
 
test_df = pd.read_csv("combined_test.csv")
test_smiles = test_df["smiles"].to_numpy()
test_labels = test_df["label"].to_numpy()
 
 
train_input = pd.read_csv('train_input.csv', header = None).to_numpy()
test_input = pd.read_csv('test_input.csv', header = None).to_numpy()

In [ ]:
# # Define the Keras TensorBoard callback.
# logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)


# my_model.fit(train_input, train_labels, epochs=10, verbose=2, validation_split=0.1, batch_size=32, callbacks=[tensorboard_callback])

my_model.fit(train_input, train_labels, epochs=10, verbose=2, validation_split=0.1, batch_size=47)

Epoch 1/10
9/9 - 0s - loss: 0.2495 - accuracy: 0.5394 - val_loss: 0.2499 - val_accuracy: 0.5709
Epoch 2/10
9/9 - 0s - loss: 0.2496 - accuracy: 0.5373 - val_loss: 0.2500 - val_accuracy: 0.5673
Epoch 3/10
9/9 - 0s - loss: 0.2497 - accuracy: 0.5371 - val_loss: 0.2500 - val_accuracy: 0.5710
Epoch 4/10
9/9 - 0s - loss: 0.2495 - accuracy: 0.5375 - val_loss: 0.2497 - val_accuracy: 0.5746
Epoch 5/10
9/9 - 0s - loss: 0.2499 - accuracy: 0.5360 - val_loss: 0.2503 - val_accuracy: 0.5670
Epoch 6/10
9/9 - 0s - loss: 0.2497 - accuracy: 0.5369 - val_loss: 0.2501 - val_accuracy: 0.5670
Epoch 7/10
9/9 - 0s - loss: 0.2497 - accuracy: 0.5358 - val_loss: 0.2495 - val_accuracy: 0.5746
Epoch 8/10
9/9 - 0s - loss: 0.2497 - accuracy: 0.5382 - val_loss: 0.2493 - val_accuracy: 0.5773
Epoch 9/10
9/9 - 0s - loss: 0.2496 - accuracy: 0.5374 - val_loss: 0.2501 - val_accuracy: 0.5671
Epoch 10/10
9/9 - 0s - loss: 0.2497 - accuracy: 0.5366 - val_loss: 0.2499 - val_accuracy: 0.5670


In [ ]:
%tensorboard --logdir logs

In [ ]:
test_scores = my_model.evaluate(test_input, test_labels, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

7/7 - 0s - loss: 0.2588 - accuracy: 0.4433
Test loss: 0.258788138628006
Test accuracy: 0.443303644657135
